In [ ]:
pip install streamlit opencv-python-headless numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.9 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import cv2
import numpy as np
from datetime import datetime
import os

# Load the Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def detect_faces(image, scale_factor, min_neighbors):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=min_neighbors)

    return faces

def draw_faces(image, faces, rect_color):
    # Convert hex color to RGB
    hex_color = rect_color.lstrip('#')
    rgb_color = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    rgb_color = (rgb_color[2], rgb_color[1], rgb_color[0])  # Convert to BGR for OpenCV

    # Draw rectangles around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), rgb_color, 2)

    return image

def main():
    st.title("Face Detection App with Viola-Jones Algorithm")

    # Instructions
    st.markdown("""
    ### Instructions
    1. **Upload an Image**: Click the file uploader below to select an image (JPG, PNG, or JPEG).
    2. **Adjust Parameters**:
       - Use the **Scale Factor** slider to control the image scaling step (1.1 to 2.0). Lower values are more precise but slower.
       - Use the **Min Neighbors** slider to set the minimum number of neighboring rectangles (0 to 10). Higher values reduce false positives.
       - Choose the **Rectangle Color** to customize the color of the face detection rectangles.
    3. **View Results**: The app will display the image with detected faces.
    4. **Save Image**: Click the "Save Image" button to download the processed image with detected faces.
    """)

    # File uploader
    uploaded_file = st.file_uploader("Upload an image", type=["jpg", "png", "jpeg"])

    if uploaded_file is not None:
        # Read the image
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

        # Display original image
        st.image(image, caption="Uploaded Image", use_column_width=True, channels="BGR")

        # Parameter sliders
        scale_factor = st.slider("Scale Factor", min_value=1.1, max_value=2.0, value=1.3, step=0.1)
        min_neighbors = st.slider("Min Neighbors", min_value=0, max_value=10, value=5, step=1)

        # Color picker for rectangles
        rect_color = st.color_picker("Rectangle Color", "#00FF00")  # Default: Green

        # Detect faces
        faces = detect_faces(image, scale_factor, min_neighbors)

        # Draw rectangles on the image
        output_image = draw_faces(image.copy(), faces, rect_color)

        # Display the result
        st.image(output_image, caption=f"Detected {len(faces)} Face(s)", use_column_width=True, channels="BGR")

        # Save image feature
        if st.button("Save Image"):
            # Generate unique filename with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_filename = f"detected_faces_{timestamp}.jpg"

            # Save the image
            cv2.imwrite(output_filename, output_image)
            st.success(f"Image saved as {output_filename}")

            # Provide download link
            with open(output_filename, "rb") as file:
                st.download_button(
                    label="Download Processed Image",
                    data=file,
                    file_name=output_filename,
                    mime="image/jpeg"
                )

if __name__ == "__main__":
    # Check if Haar Cascade file exists
    if not os.path.exists('haarcascade_frontalface_default.xml'):
        st.error("Error: haarcascade_frontalface_default.xml not found. Please ensure the file is in the same directory.")
    else:
        main()

2025-10-26 19:31:20.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 19:31:20.093 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 19:31:20.095 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
